<a href="https://colab.research.google.com/github/dssg/triage/blob/kit_colab_triage/example/colab/colab_triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Triage

This notebook provides a quick, interactive tutorial for `triage`, a python machine learning pipeline for social good problems.

For the tutorial, we'll be using a sample of the data provided by DonorsChoose to the 2014 KDD Cup -- the full dataset and some details about it can be found [here](https://www.kaggle.com/c/kdd-cup-2014-predicting-excitement-at-donors-choose/data). DonorsChoose is a crowdfunding platform for teachers to seek funding for projects and resources from the community. Like other crowdfunding sites, a project is only funded if it reaches its goal (a project will expire if it hasn't been fully funded after 4 months).

For our purposes here, we imagine that donors choose has a new program that will help projects at risk of not being fully funded by providing a review from a digital content expert. Our job is to help them identify those projects.



## Getting Set Up

We'll need a few dependencies to run triage in a colab notebook:
- A local postgresql server (we'll use version 11)
- A simplified dataset loaded into this database (we'll use data from DonorsChoose)
- Triage and its dependencies (we'll use the current version in pypi)

In [1]:
# Install and start postgresql-11 server
!sudo apt-get -y -qq update
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ `lsb_release -cs`-pgdg main" |sudo tee  /etc/apt/sources.list.d/pgdg.list
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-11 postgresql-client-11
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=postgres
%env DEMO_DATABASE_PASS=postgres

OK
deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 16.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../01-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../02-netbase_5.4_all.deb

In [2]:
# Download sampled DonorsChoose data and load it into our postgres server
!curl -s -OL https://dsapp-public-data-migrated.s3.us-west-2.amazonaws.com/donors_sampled_20210920_v3.dmp
!PGPASSWORD=$DEMO_DATABASE_PASS pg_restore -h $DEMO_DATABASE_HOST -p $DEMO_DATABASE_PORT -d $DEMO_DATABASE_NAME -U $DEMO_DATABASE_USER -O -j 8 donors_sampled_20210920_v3.dmp

In [3]:
# Install triage and its dependencies
!pip install triage

     |████████████████████████████████| 250 kB 27.9 MB/s 
     |████████████████████████████████| 227 kB 73.1 MB/s 
     |████████████████████████████████| 636 kB 53.0 MB/s 
     |████████████████████████████████| 61 kB 380 kB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 129 kB 57.3 MB/s 
     |████████████████████████████████| 2.9 MB 55.7 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.2 MB 29.5 MB/s 
     |████████████████████████████████| 1.3 MB 32.3 MB/s 
     |████████████████████████████████| 25.9 MB 38.4 MB/s 
     |████████████████████████████████| 15.7 MB 34.4 MB/s 
     |████████████████████████████████| 6.8 MB 44.1 MB/s 
     |████████████████████████████████| 10.1 MB 31.2 MB/s 
     |████████████████████████████████| 215 kB 65.1 MB/s 
     

🛑  &nbsp;&nbsp;**NOTE: Before continuing, your colab runtime may need to be restarted for the installed packages to take effect. If a "Restart Runtime" button appeared at the bottom of the output above, be sure to click it before moving on to the next section!**

## A Quick Look at the DonorsChoose Data

Before getting into triage, let's just take a quick look at the data we'll be using here. To get started, we'll need to connect to the database we just created...

In [1]:
from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
import pandas as pd

db_url = URL(
            'postgres',
            host='localhost',
            username='postgres',
            database='donors_choose',
            password='postgres',
            port=5432,
        )

db_engine = create_engine(db_url)

The DonorsChoose dataset contains four main tables we'll need here:
- **Projects** contains information about each project as well as some details about the teacher posting it and their school and district
- **Essays** contains the detailed descriptions that the teacher post describing their project and needs
- **Resources** contains detailed information about the specific number, type, and cost of resources being asked for in the project
- **Donations** contains information about the donations received by each project on a transactional level, as well as some details about the donor

Let's take a look at the projects:

In [2]:
pd.read_sql('SELECT COUNT(*) FROM data.projects', db_engine)

,count
0,16480


In [3]:
pd.read_sql('SELECT * FROM data.projects LIMIT 5', db_engine)

,schoolid,projectid_str,teacher_acctid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_asking_price,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,entity_id
0,29a2da790e38b6c8a1c70aced6b9c765,30c034618e67d00c641f9b5b7775c0f4,0903da60e148adc6280d55f5d94791a5,192013001182,41.428391,-91.049135,Muscatine,IA,52761,None,Muscatine Cmty School District,Muscatine,False,False,False,False,False,False,Ms.,False,False,Applied Sciences,Math & Science,College & Career Prep,Applied Learning,Supplies,high poverty,Grades 6-8,35.0,511.32,601.55,100,False,False,2012-08-06,234148
1,06ef48acbdf9b013d4bc4bfc8d328cc1,94199c544d9d2926c1820e5c6cde1eb6,fb340c7ac3b22a5984c6a82602e4a510,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Ms.,False,False,Literacy,Literacy & Language,ESL,Literacy & Language,Supplies,high poverty,Grades 3-5,35.0,167.43,192.45,40,False,False,2010-07-10,453579
2,06ef48acbdf9b013d4bc4bfc8d328cc1,61f42f174afef1ed2419ab948a647137,fb340c7ac3b22a5984c6a82602e4a510,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Ms.,False,False,Literacy,Literacy & Language,Applied Sciences,Math & Science,Other,high poverty,Grades 3-5,35.0,167.43,196.98,45,False,False,2011-06-09,353855
3,06ef48acbdf9b013d4bc4bfc8d328cc1,c966f5226f42aaaf6b115d7dbaefdea3,e0b5a72f41a376b28db9c2e838a24de5,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Mrs.,False,False,Literacy,Literacy & Language,Health & Life Science,Math & Science,Other,high poverty,Grades 3-5,35.0,162.14,190.75,25,False,False,2011-06-13,353178
4,06ef48acbdf9b013d4bc4bfc8d328cc1,69e9c24111daa1e6ba64c6d5538e4df1,32e86c49396707f71fdc0398ab2b844b,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Mrs.,False,False,History & Geography,History & Civics,Environmental Science,Math & Science,Books,high poverty,Grades 3-5,35.0,381.27,448.55,25,False,False,2012-07-15,239363


Note that the `projectid_str` column can be used to link out to the other tables. For instance, let's look at what we can find out about project `30c034618e67d00c641f9b5b7775c0f4`:

In [10]:
pd.read_sql("SELECT * FROM data.essays WHERE projectid_str='30c034618e67d00c641f9b5b7775c0f4'", db_engine)

,projectid_str,teacher_acctid,title,short_description,need_statement,essay,entity_id,date_posted
0,30c034618e67d00c641f9b5b7775c0f4,0903da60e148adc6280d55f5d94791a5,Illuminate Your Future,Can you imagine what the future will bring wit...,My students need LED bulbs and electronic comp...,Can you imagine what the future will bring wit...,234148,2012-08-06


In [19]:
pd.read_sql("""
  SELECT project_resource_type,
        COUNT(*) AS num_distinct_resources, 
        SUM(item_quantity) AS num_total_resources,
        AVG(item_unit_price) AS avg_price,
        SUM(item_unit_price * item_quantity) AS total_cost
  FROM data.resources 
  WHERE projectid_str='30c034618e67d00c641f9b5b7775c0f4'
  GROUP BY 1;
  """, db_engine)

,project_resource_type,num_distinct_resources,num_total_resources,avg_price,total_cost
0,Supplies,7,113.0,13.004286,400.19


In [21]:
pd.read_sql("""
  SELECT 
    COUNT(*) AS num_donations,
    SUM(donation_to_project) AS total_donation,
    SUM(CASE WHEN is_teacher_acct THEN 1 ELSE 0 END) AS num_teacher_donation
  FROM data.donations 
  WHERE projectid_str='30c034618e67d00c641f9b5b7775c0f4'
  ;
  """, db_engine)

,num_donations,total_donation,num_teacher_donation
0,1,511.32,0


## Formulating the project

Now that we're familiar with the available data, let's turn to the prediction problem at hand. Because reviewing and offering suggestions to posted projects will be time and resource-intensive, we might assume that DonorsChoose can only help a fraction of all projects that get posted, let's suppose 10%. Then, we might formulate our problem along the lines of:

**Each day, for all the projects posted on that day, can we identify the 10% of projects with the highest risk of not being fully funded within 4 months to prioritize for review by digital content experts.**

With this formulation in mind, we can define a cohort and label for our analysis. `triage` will allow us to define these directly as a SQL query, so let's start there...

### Defining the Cohort

Because most models to inform important decisions will need to generalize into the future, `triage` focuses on respecting the temporal nature of the data (discussed in more detail below). The `cohort` is the set of relevant entities for model training/prediction at a given point in time, which `triage` referrs to as an `as_of_date`.

🚧 &nbsp;&nbsp;NOTE: In `triage`, an `as_of_date` is taken to be midnight at the **beginning** of that date.

Here, the cohort is relatively straightforward: we simply want to identify all of the projects that were posted, right on the day of posting. Although we were looking at the identifier `projectid_str` above, `triage` looks for a column called `entity_id` to uniquely identify entities to its models. We've already added this column to this dataset, so we'll use that below.

🚧 &nbsp;&nbsp;NOTE: `triage` expects entities in the data to be identified by an **integer column** called `entity_id`.

With those details in mind, let's look at an example of how we might define the cohort from our data for this project:

In [23]:
example_as_of_date = '2012-08-07'

pd.read_sql("""
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
  ;
  """.format(as_of_date=example_as_of_date), db_engine)

,entity_id
0,234035
1,234148
2,234234


In `triage` we'll be able to use `{as_of_date}` as a placeholder for time just as we're doing here.

Also note that because the `as_of_date` is taken to be midnight, we're looking at the projects posted the previous day (hence subtracting the 1 day interval in the query).

For `triage`, we use a yaml format for configuration (described further below) and we'll be able to provide this query directly:
```
cohort_config:
  query: |
    SELECT distinct(entity_id)
      FROM data.projects
    WHERE date_posted = '{as_of_date}'::date - interval '1day'
```

### Defining the Label

For modeling, we also need to consider the outcome we care about. Returning to our formulation, we described trying to identify projects which will not be fully funded within the four months they are active on the platform.

As with the cohort, notice that labels are calculated relative to a given point in time (the `as_of_date` described above) and over a specific time horizon (here, 4 months from posting). In triage, this time horizon is referred to as a `label_timespan` and is also available as a parameter to your label definition, again specified as a query:

In [28]:
example_as_of_date = '2012-08-07'
example_label_timespan = '4month'

pd.read_sql("""
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT 
        c.entity_id, 
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d 
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp 
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE 
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome  
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)
  ;
  """.format(as_of_date=example_as_of_date, label_timespan=example_label_timespan), db_engine)

,entity_id,outcome
0,234035,1
1,234148,0
2,234234,0


A little more complicated than our cohort query, but still reasonably straightforward: we start with the cohort defined above, then find all the donations to those projects within the label timespan (e.g., the following 4 months after posting), and finally compare that to the total price of the project to create a binary classification label for whether or not the project was fully funded.

Notice here that because we will intervene on projects at risk for **NOT** being fully funded, we define this as our class 1 label while those that do reach their funding goal are given class 0.

As with the cohort, we'll be able to specify this label query directly to triage in our yaml configuation:
```
label_config:
  query: |
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT 
        c.entity_id, 
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d 
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp 
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE 
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome  
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)

  name: 'fully_funded'
```

For more details these two pieces of the modeling pipeline, see the [cohort and label deep dive in the triage docs](https://dssg.github.io/triage/experiments/cohort-labels/). 

### Dealing with Time

As noted above, `triage` is designed for problems where the desire to generalize to future data and therefore is careful to respect the temporal nature of the problem. This is particularly salient in two places: defining the validation strategy for model evaluation and ensuring that features only make use of information available at the time of analysis/prediction.

For validation, the idea is generally simple: models should be trained on historical data and validated on future data. As such, `triage` constructs validation splits that reflect this process by using a certain point in time as the cut-off between training and validation and then moving this cut-off back through the data to generate multiple splits. The implementation is a bit more complicated and relies on several parameters, the details of which we won't go deep into here, but you can find a much deeper discussion in the [longer "dirty duck" tutorial](https://dssg.github.io/triage/dirtyduck/triage_intro/) as well as in the [experiment config docs](https://dssg.github.io/triage/experiments/experiment-config/).

![temporal figure](https://dssg.github.io/triage/experiments/temporal_config_graph.png)

In short, these parameters are (illustrated across three training/validation splits in the figure above):
- feature start/end times: what range of history is feature information available for?
- label start/end times: what range of history is outcome (label) data available for?
- model update frequency: what is the interval between refreshes of the model?
- test durations: over what time period will the model be in use for making predictions?
- max training history: how much historical data should be used for model training (that is, for rows/examples)?
- training/test as_of_date frequencies: within a training or validation (test) set, how frequently should cohorts be sampled?
- training/test label timespans: over what time horizon are labels (outcomes) collected?

As with the cohorts and labels, these parameters are specified to `triage` via its yaml configuration file. Here's what this will look like for our setting:
```
temporal_config:

    # first date our feature data is good
    feature_start_time: '2000-01-01'
    feature_end_time: '2013-06-01'

    # first date our label data is good
    # donorschoose: as far back as we have good donation data
    label_start_time: '2011-09-02'
    label_end_time: '2013-06-01'

    model_update_frequency: '4month'

    # length of time defining a test set
    test_durations: ['3month']
    # defines how far back a training set reaches
    max_training_histories: ['1y']

    # we sample every day, since new projects are posted
    # every day
    training_as_of_date_frequencies: ['1day']
    test_as_of_date_frequencies: ['1day']
    
    # when posted project timeout
    label_timespans: ['3month']
```

### Model Evaluation Metrics

### Defining Features

### Model and Hyperparameter Grid

## experiment_config.yaml

In [ ]:
config_yaml = """
config_version: 'v7'

model_comment: 'triage demo'

random_seed: 1995

temporal_config:

    # first date our feature data is good
    feature_start_time: '2000-01-01'
    feature_end_time: '2013-06-01'

    # first date our label data is good
    # donorschoose: as far back as we have good donation data
    label_start_time: '2011-09-02'
    label_end_time: '2013-06-01'

    model_update_frequency: '4month'

    # length of time defining a test set
    test_durations: ['3month']
    # defines how far back a training set reaches
    max_training_histories: ['1y']

    # we sample every day, since new projects are posted
    # every day
    training_as_of_date_frequencies: ['1day']
    test_as_of_date_frequencies: ['1day']
    
    # when posted project timeout
    label_timespans: ['3month']
    

cohort_config:
  query: |
    SELECT distinct(entity_id)
      FROM data.projects
    WHERE date_posted = '{as_of_date}'::date - interval '1day'

label_config:
  query: |
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT 
        c.entity_id, 
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d 
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp 
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE 
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome  
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)

  name: 'fully_funded'


feature_aggregations:
  -
    prefix: 'project_features'
    from_obj: 'data.projects'
    knowledge_date_column: 'date_posted'

    aggregates_imputation:
      all:
        type: 'zero'

    categoricals_imputation:
      all:
        type: 'null_category'          

    categoricals:
      -
        column: 'resource_type'
        metrics:
          - 'max' 
        choice_query: 'select distinct resource_type from data.projects'
    
    aggregates:
      -
        quantity: 'total_asking_price'
        metrics:
          - 'sum'
      
    # Since our time-aggregate features are precomputed, feature interval is 
    # irrelvant. We keep 'all' as a default.
    intervals: ['all'] 
    groups: ['entity_id']

grid_config:
    'sklearn.ensemble.RandomForestClassifier':
      n_estimators: [150]
      max_depth: [50]
      min_samples_split: [25]
    
    'sklearn.tree.DecisionTreeClassifier':
      max_depth: [3]
      max_features: [null]
      min_samples_split: [25]
      
    'triage.component.catwalk.estimators.classifiers.ScaledLogisticRegression':
        C: [0.1]
        penalty: ['l1']
    
    'triage.component.catwalk.baselines.rankers.PercentileRankOneFeature':
      feature: ['project_features_entity_id_all_total_asking_price_sum']
      descend: [False]


scoring:
    testing_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]

    training_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]
          
bias_audit_config:
    from_obj_table: 'data.projects'
    attribute_columns:
      - 'teacher_prefix'
    knowledge_date_column: 'date_posted'
    entity_id_column: 'entity_id'
    ref_groups_method: 'predefined'
    ref_groups:
        'teacher_prefix': 'Mr.'
    thresholds:
        percentiles: [5, 10, 15, 20, 25, 50, 100]
        top_n: [25, 50, 100]

individual_importance:
    methods: [] # empty list means don't calculate individual importances
    n_ranks: 1 
"""

## database.yaml

In [ ]:
database_yaml = """
host: localhost
user: postgres
db: donors_choose
pass: postgres
port: 5432
role: postgres
"""

## run.py

In [ ]:
import yaml

from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
from triage.experiments import MultiCoreExperiment
import logging

import os

from sqlalchemy.event import listens_for
from sqlalchemy.pool import Pool

def run_triage():

  # andrew_id = os.getenv('USER')
  # user_path = os.path.join('/data/users/', andrew_id)
  user_path = '/content'

  # add logging to a file (it will also go to stdout via triage logging config)
  log_filename = os.path.join(user_path, 'triage.log')
  logger = logging.getLogger('')
  hdlr = logging.FileHandler(log_filename)
  hdlr.setLevel(15)   # verbose level
  hdlr.setFormatter(logging.Formatter('%(name)-30s  %(asctime)s %(levelname)10s %(process)6d  %(filename)-24s  %(lineno)4d: %(message)s', '%d/%m/%Y %I:%M:%S %p'))
  logger.addHandler(hdlr)

  # creating database engine
  # dbfile = os.path.join(user_path, 'database.yaml')

  # with open(dbfile, 'r') as dbf:
  #     dbconfig = yaml.safe_load(dbf)

  dbconfig = yaml.safe_load(database_yaml)
  print(dbconfig['role'])

  # assume group role to ensure shared permissions
  @listens_for(Pool, "connect")
  def assume_role(dbapi_con, connection_record):
      logging.debug(f"setting role {dbconfig['role']};")
      dbapi_con.cursor().execute(f"set role {dbconfig['role']};")
      # logging.debug(f"setting role postres;")
      # dbapi_con.cursor().execute(f"set role postgres;")

  db_url = URL(
              'postgres',
              host=dbconfig['host'],
              username=dbconfig['user'],
              database=dbconfig['db'],
              password=dbconfig['pass'],
              port=dbconfig['port'],
          )

  db_engine = create_engine(db_url)

  triage_output_path = os.path.join(user_path, 'triage_output')
  os.makedirs(triage_output_path, exist_ok=True)

  # loading config file
  # with open('%s_triage_config.yaml' % andrew_id, 'r') as fin:
  #     config = yaml.safe_load(fin)

  config = yaml.safe_load(config_yaml)

  # creating experiment object
  experiment = MultiCoreExperiment(
      config = config,
      db_engine = db_engine,
      project_path = triage_output_path,
      n_processes=2,
      n_db_processes=2,
      replace=True,
      save_predictions=True
      )

  # experiment.validate()
  experiment.run()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Let's run triage!

In [ ]:
run_triage()

postgres
2021-12-07 22:26:22 - VERBOSE Matrices and trained models will be saved in /content/triage_output
2021-12-07 22:26:22 -  NOTICE Replace flag is set to true. Matrices, models, evaluations and predictions (if exist) will be replaced
2021-12-07 22:26:22 -    INFO No results_schema_versions table exists, which means that this installation is fresh. Upgrading db.
2021-12-07 22:26:22 -    INFO Context impl PostgresqlImpl.
2021-12-07 22:26:22 -    INFO Will assume transactional DDL.
2021-12-07 22:26:22 -    INFO Running upgrade  -> 8b3f167d0418, empty message
2021-12-07 22:26:22 -    INFO Running upgrade 8b3f167d0418 -> 0d44655e35fd, empty message
2021-12-07 22:26:22 -    INFO Running upgrade 0d44655e35fd -> 264245ddfce2, empty message
2021-12-07 22:26:22 -    INFO Running upgrade 264245ddfce2 -> 72ac5cbdca05, Change importance to float
2021-12-07 22:26:22 -    INFO Running upgrade 72ac5cbdca05 -> 7d57d1cf3429, empty message
2021-12-07 22:26:22 -    INFO Running upgrade 7d57d1cf3429 

## Checking the results

Confirming we got some models...

In [ ]:
import yaml
from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
import pandas as pd

dbconfig = yaml.safe_load(database_yaml)
db_url = URL(
            'postgres',
            host=dbconfig['host'],
            username=dbconfig['user'],
            database=dbconfig['db'],
            password=dbconfig['pass'],
            port=dbconfig['port'],
        )

db_engine = create_engine(db_url)

In [ ]:
pd.read_sql('SELECT * FROM triage_metadata.models;', db_engine)

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,hyperparameters,model_comment,batch_comment,config,train_end_time,test,train_matrix_uuid,training_label_timespan,model_size,random_seed,built_in_triage_run
0,1,2,395896e2f4506e13d4ebe536c72f4d81,2021-12-07 22:27:22.721503,2021-12-07 22:27:22.344349,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 3, 'max_features': None, 'min_sa...",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,908907174,1
1,2,3,58b2330d957c967f3e9de50a09eca3da,2021-12-07 22:27:22.728391,2021-12-07 22:27:22.344349,triage.component.catwalk.estimators.classifier...,"{'C': 0.1, 'penalty': 'l1'}",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,1656233507,1
2,3,4,fae0c1a55e7c9bd9fa9064b2cd481371,2021-12-07 22:27:52.231542,2021-12-07 22:27:22.344349,triage.component.catwalk.baselines.rankers.Per...,"{'descend': False, 'feature': 'project_feature...",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,1259133573,1
3,4,2,232a3978ccf34b167884daf702e8ed26,2021-12-07 22:27:52.840186,2021-12-07 22:27:22.402445,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 3, 'max_features': None, 'min_sa...",triage demo,None,None,2012-08-01,False,10f30faf4d136757ef3f11d9b3e5bdc2,90 days,0.0625,1106414652,1
4,5,3,b21e79f1c30cf98f035f493fd8dae06c,2021-12-07 22:28:01.561102,2021-12-07 22:27:22.402445,triage.component.catwalk.estimators.classifier...,"{'C': 0.1, 'penalty': 'l1'}",triage demo,None,None,2012-08-01,False,10f30faf4d136757ef3f11d9b3e5bdc2,90 days,0.0625,897217774,1
5,6,4,dd806432f4bfd195aade5a559cf076d2,2021-12-07 22:28:28.436616,2021-12-07 22:27:22.402445,triage.component.catwalk.baselines.rankers.Per...,"{'descend': False, 'feature': 'project_feature...",triage demo,None,None,2012-08-01,False,10f30faf4d136757ef3f11d9b3e5bdc2,90 days,0.0625,1431439151,1
6,7,2,4b37802241efcfae1e82485fb3f48ef9,2021-12-07 22:28:37.678829,2021-12-07 22:27:22.447742,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 3, 'max_features': None, 'min_sa...",triage demo,None,None,2012-12-01,False,f8772c9ca6484d2f32a7e8258f6dbc64,90 days,0.0625,1463730397,1
7,8,3,21c8665d4b12f8f095c90dd0bf0d5f68,2021-12-07 22:28:38.689047,2021-12-07 22:27:22.447742,triage.component.catwalk.estimators.classifier...,"{'C': 0.1, 'penalty': 'l1'}",triage demo,None,None,2012-12-01,False,f8772c9ca6484d2f32a7e8258f6dbc64,90 days,0.0625,1879462244,1
8,9,4,3510f611b7ec6e3c4751ba0f81170d63,2021-12-07 22:29:15.286520,2021-12-07 22:27:22.447742,triage.component.catwalk.baselines.rankers.Per...,"{'descend': False, 'feature': 'project_feature...",triage demo,None,None,2012-12-01,False,f8772c9ca6484d2f32a7e8258f6dbc64,90 days,0.0625,827031307,1
9,10,1,afe9d61f915aed6702fc598ceb66fdaf,2021-12-07 22:29:22.295662,2021-12-07 22:27:22.344349,sklearn.ensemble.RandomForestClassifier,"{'max_depth': 50, 'n_estimators': 150, 'min_sa...",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,1443952767,1


Confirming we saved predictions...

In [ ]:
pd.read_sql('SELECT * FROM test_results.predictions LIMIT 5;', db_engine)

,model_id,entity_id,as_of_date,score,label_value,rank_abs_no_ties,rank_abs_with_ties,rank_pct_no_ties,rank_pct_with_ties,matrix_uuid,test_label_timespan
0,3,254981,2012-04-12,0.99742,0,1,1,0.00258,0.00259,2a2ba0ccc318bbe74a99726a72c323b0,90 days
1,3,249195,2012-05-12,0.99485,1,2,2,0.00515,0.00518,2a2ba0ccc318bbe74a99726a72c323b0,90 days
2,3,255422,2012-04-09,0.99227,0,3,3,0.00773,0.00777,2a2ba0ccc318bbe74a99726a72c323b0,90 days
3,3,253874,2012-04-18,0.98969,0,4,4,0.01031,0.01036,2a2ba0ccc318bbe74a99726a72c323b0,90 days
4,3,250376,2012-05-09,0.98711,0,5,5,0.01289,0.01295,2a2ba0ccc318bbe74a99726a72c323b0,90 days


Location of triage outputs...

In [ ]:
!ls triage_output/

matrices  trained_models


In [ ]:
!ls -la triage_output/matrices/


total 128
drwxr-xr-x 2 root root  4096 Dec  7 22:27 .
drwxr-xr-x 4 root root  4096 Dec  7 22:27 ..
-rw-r--r-- 1 root root 17352 Dec  7 22:27 10f30faf4d136757ef3f11d9b3e5bdc2.csv.gz
-rw-r--r-- 1 root root  7029 Dec  7 22:27 10f30faf4d136757ef3f11d9b3e5bdc2.yaml
-rw-r--r-- 1 root root  3950 Dec  7 22:27 2a2ba0ccc318bbe74a99726a72c323b0.csv.gz
-rw-r--r-- 1 root root  3369 Dec  7 22:27 2a2ba0ccc318bbe74a99726a72c323b0.yaml
-rw-r--r-- 1 root root  6136 Dec  7 22:27 76c9a74b579fcdc3747e18fdfa286e35.csv.gz
-rw-r--r-- 1 root root  3345 Dec  7 22:27 76c9a74b579fcdc3747e18fdfa286e35.yaml
-rw-r--r-- 1 root root  8953 Dec  7 22:27 88033d4d2e731553062324d7403af43f.csv.gz
-rw-r--r-- 1 root root  4125 Dec  7 22:27 88033d4d2e731553062324d7403af43f.yaml
-rw-r--r-- 1 root root 10533 Dec  7 22:27 cc681ad650fafd044897af6bd73ab167.csv.gz
-rw-r--r-- 1 root root  3393 Dec  7 22:27 cc681ad650fafd044897af6bd73ab167.yaml
-rw-r--r-- 1 root root 23340 Dec  7 22:27 f8772c9ca6484d2f32a7e8258f6dbc64.csv.gz
-rw-r--r-